# Sensitivity Analysis with Custom, Complex Multi-Variate Distributions

Notebook developed by Banamali Panigrahi and Saman Razavi 

## For the X-VARS method, please cite: 
Panigrahi, B., Razavi, S., Doig, L. E., Cordell, B., Gupta, H. V., & Liber, K. (2024). On Robustness of the Explanatory Power of Machine Learning Models: Insights from a New Explainable AI Approach using Sensitivity Analysis. Water Resources Research, XX(XXX), e2024WR0XXXX. https://doi.org/10.XXXX/2024WR0XXXX

# Example: SA of the Machine Learning Models When Inputs Follow Complex Distribution

## Objective:
This notebook runs a sensitivity analysis when the theoretical distribution of some inputs is unknown or too complex. Here, the G-VARS (By Do and razavi, 2020) is further extended to accommodate any complex distribution of data that often encountered with environmental systems.

### Key Points
(1) accounts for complex multi-variate distributional properties of the input-output data, commonly observed with environmental systems
(2) offers a global assessment of the input-output response surface formed by machine learning (ML), rather than focusing solely on local regions around existing data points, and 
(3) is scalable and independent of data size, providing computational efficiency when dealing with large datasets

Here, for demonstration purposes,the X-VARS test function for Light gradient boosting (LgBoost) model is used. But to answer the above questions, it can be replaced with any other machine-learning models.

First import the required libraries, including GVARS and machine learning model libraries for creating a wrapper around the desired model.    

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from varstool import GVARS, Model
from pandas.core.frame import DataFrame 

## Libraries for all connectionist, Kernel, and Tree based ML models

In [ ]:
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import StandardScaler
from keras import Sequential
from keras.layers import Dense
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from lightgbm import LGBMRegressor
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from matplotlib import pyplot
from keras.callbacks import EarlyStopping
from keras.regularizers import l1
from keras.regularizers import l2
from tensorflow.keras.layers import Dropout
from sklearn.metrics import r2_score
from sklearn import metrics
from sklearn.model_selection import cross_val_score
import random

In [ ]:
#Set Seed numbers
random.seed(124321)
np.random.seed(124321)
tf.random.set_seed(124321)

# Preapare the input-target dataset matrix and load data in .CSV format

In [ ]:
# The dataset in the .csv file are arranged so as to keep the target in the last column
Data = pd.read_csv('path/test_data.csv')                                                                                                

In [ ]:
#Here inputs are from o to 12th Column and 13th colum is the output  
X = Data.iloc[:, 0:13].to_numpy()
y = Data.iloc[:, 13].to_numpy().reshape(-1, 1)
Data

In [ ]:
# Calculate correlation matrix
#correlation_matrix = Data.corr()

In [ ]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30)

In [ ]:
print("Shape of X_train: ",X_train.shape)
print("Shape of X_test: ", X_test.shape)
print("Shape of y_train: ",y_train.shape)
print("Shape of y_test: ",y_test.shape)

In [ ]:
sc_X = StandardScaler()
sc_y = StandardScaler()
X_scaled = sc_X.fit_transform(X_train)
y_scaled = sc_y.fit_transform(y_train.reshape(-1,1))

X_scaled_test = sc_X.fit_transform(X_test)
y_scaled_test = sc_y.fit_transform(y_test.reshape(-1,1))

In [ ]:
# Fitting the Model to the training dataset
regressor_lgboost = LGBMRegressor()
regressor_lgboost.fit(X_scaled, y_scaled.ravel())

In [ ]:
# lgboost model is used for prediction of the target variable using the train and test set
# And converting to original data for calculating the prediction performance metrices

y_pred_train = sc_y.inverse_transform(regressor_lgboost.predict(X_scaled).reshape(-1, 1))
y_pred_test = sc_y.inverse_transform(regressor_lgboost.predict(X_scaled_test).reshape(-1, 1))

# Prediction scores for the Train and Test set results
r2_score_train = r2_score(y_train, y_pred_train)
r2_score_test = r2_score(y_test, y_pred_test)

# Predicting RMSE 
rmse_train = (np.sqrt(mean_squared_error(y_train, y_pred_train)))
rmse_test = (np.sqrt(mean_squared_error(y_test, y_pred_test)))

# Predicting MSE 
mse_train = (mean_squared_error(y_train, y_pred_train))
mse_test = (mean_squared_error(y_test, y_pred_test))

# Predicting MAE 
mae_train = metrics.mean_absolute_error(y_train, y_pred_train)
mae_test = metrics.mean_absolute_error(y_test, y_pred_test)

print('R2_score (train): ', r2_score_train)
print('R2_score (test): ', r2_score_test)
print("RMSE (train): ", rmse_train)
print("RMSE (test): ", rmse_test)
print("MSE (train): ", mse_train)
print("MSE (test): ", mse_test)
print("MAE (train): ", mae_train)
print("MAE (test): ", mae_test)

# Introduce the LgBoost model function for sensitivity analysis

In [ ]:
#a model is defined to apply the trained/fitted lgboost model with the available dataset callable inside the XVARS framework
def LGBMRegressorTest(x):
    x_scaled = sc_X.transform(np.array(x).reshape(1, -1))
    y_pred_lgboost = regressor_lgboost.predict(x_scaled)
    y_pred_lgboost = y_pred_lgboost.astype(float)
    y_pred_original = sc_y.inverse_transform(y_pred_lgboost.reshape(1, -1))
    return y_pred_original[0, 0]

In [ ]:
lgboost_model = Model(LGBMRegressorTest)

In [ ]:
x=pd.Series({#name  #value
             'WT'   : 18.91 ,
             'WL'   : 341.72 ,
             'EC'   : 2464.8 ,
             'Turb'   : 21.59 ,
             'RF' : 7.50 ,
             'WS' : 7.0 ,
             'Rad' : 839.0 ,
             'Pres' : 60.68 ,
             'AT' : 40.30 ,
             'RH' : 100.00 ,
             'NH4' : 12.304 ,
             'Chl' : 15.60 ,
              'pH' : 8.78 ,
             })
lgboost_model(x)

# Set up a X-VARS experiment with custom distributions

Create a G-VARS experiment and set its attributes. Please refer to Do and Razavi, 2020 for better understanding.

In [ ]:
# Define Experiment 1
my_parameters = { 'WT': (None, None, None, 'custom'),
                 'WL': (None, None, None, 'custom'),
                 'EC': (None, None, None, 'custom'), 
                 'Turb': (None, None, None, 'custom'),
                 'RF': (None, None, None, 'custom'),
                 'WS': (None, None, None, 'custom'),
                 'Rad': (None, None, None, 'custom'),
                 'Press': (None, None, None, 'custom'),
                 'AT': (None, None, None, 'custom'),
                 'RH': (None, None, None, 'custom'),
                 'NH4': (None, None, None, 'custom'), 
                 'Chl': (None, None, None, 'custom'),
                 'pH': (None, None, None, 'custom'),
                  }
my_corr_mat = np.array([
                        [1.00,-0.37,0.38,-0.32,-0.04,-0.09,0.21,-0.21,0.56,-0.04,-0.27,0.18,0.40],
                        [-0.37,1.00,0.58,0.31,0.07,0.04,-0.15,0.94,-0.13,0.15,0.39,0.48,-0.96],
                        [0.38,0.58,1.00,0.00,0.06,0.01,0.10,0.68,0.37,0.02,0.19,0.54,-0.55],
                        [-0.32,0.31,0.00,1.00,0.07,0.14,-0.14,0.27,-0.23,0.17,0.29,0.06,-0.25],
                        [-0.04,0.07,0.06,0.07,1.00,0.03,-0.05,0.12,-0.03,0.07,0.10,0.01,-0.09],
                        [-0.09,0.04,0.01,0.14,0.03,1.00,0.40,0.09,0.18,-0.40,-0.02,-0.11,-0.03],
                        [0.21,-0.15,0.10,-0.14,-0.05,0.40,1.00,-0.03,0.59,-0.67,-0.14,-0.13,0.13],
                        [-0.21,0.94,0.68,0.27,0.12,0.09,-0.03,1.00,0.07,0.07,0.37,0.54,-0.91],
                        [0.56,-0.13,0.37,-0.23,-0.03,0.18,0.59,0.07,1.00,-0.65,-0.21,0.10,0.18],
                        [-0.04,0.15,0.02,0.17,0.07,-0.40,-0.67,0.07,-0.65,1.00,0.24,0.13,-0.13],
                        [-0.27,0.39,0.19,0.29,0.10,-0.02,-0.14,0.37,-0.21,0.24,1.00,0.08,-0.39],
                        [0.18,0.48,0.54,0.06,0.01,-0.11,-0.13,0.54,0.10,0.13,0.08,1.00,-0.51],
                        [0.40,-0.96,-0.55,-0.25,-0.09,-0.03,0.13,-0.91,0.18,-0.13,-0.39,-0.51,1.00],
                       ])
my_num_dir_samples = 10
my_delta_h = 0.1 # my_delta_h = 1 / my_num_dir_samples or choose values such as 0.1

# Define the Experiment

In [ ]:
experiment_1 = GVARS(parameters      = my_parameters,
                    corr_mat         = my_corr_mat,
                    num_stars        = 100,
                    num_dir_samples  = my_num_dir_samples,
                    delta_h          = my_delta_h,
                    ivars_scales     = (0.1, 0.3, 0.5),
                    sampler          = 'lhs',
                    slice_size       = 10,
                    model            = lgboost_model,
                    seed             = 123456789,
                    bootstrap_flag   = True,
                    bootstrap_size   = 100,
                    bootstrap_ci     = 0.9,
                    grouping_flag    = False,
                    num_grps         = 2,
                    dist_sample_file = 'path/test_data.csv',
                    fictive_mat_flag = True,
                    report_verbose   = True,
                    )

# Run the X-VARS experiments

In [ ]:
experiment_1.run_online()

In [ ]:
#Display the lgboost model run output for the target variable column
#This displays the all the resampled rows i.e., 10*100*13=13000 rows for the 14 columns comprising inputs-target 
experiment_1.model_df

In [ ]:
#let's save the martix of X-VARS resampled variables
np.savetxt('path\output.csv',
           (experiment_1.model_df), delimiter=',')

# Check out the results
#When the X-VARS analysis is completed, check out the results of sensitivity analysis.

#IVARS: Integrated variogram Across a Range of Scales

#IVARS indices are the primary sensitivity indices by the VARS approach. First, print all the IVARS indices for the scale ranges (0.1 or 0.3 or 0.5) #of interest.

In [ ]:
# IVARS from Experiment 1
cols = experiment_1.parameters.keys()
experiment_1.ivars[cols]

In [ ]:
#save the IVARS values for all the inputs (to decide the ranks of the influential input parameters)
np.savetxt('path\IVARS.csv', 
           (experiment_1.ivars[cols]), delimiter=',')

In [ ]:
#Choose a scale range and plot the respective IVARS indices. Two points:

#POINT1: IVARS-50 (h=[0-0.5]), called ***Total-Variogram Effect*** is the most comprehensive sensitivity index.
#POINT2: Plotting sensitivity results in log scale helps us better differentiate less influential parameters.

In [ ]:
# Plot IVARS from Experiment 1
ivars_scale = 0.5 # Choose the scale range of interest, e.g., 0.1, 0.3, or 0.5

cols = experiment_1.parameters.keys()                     
fig_bar = plt.figure(figsize=(10,5))
plt.gca().bar(cols, experiment_1.ivars.loc[pd.IndexSlice[ ivars_scale ]][cols], color='gold')
plt.gca().set_title (r'Integrated variogram Across a Range of Scales (LgBoost_DO_124321)', fontsize = 15)
plt.gca().set_ylabel(r'IVARS-50 (Total-Variogram Effect)', fontsize = 13)
plt.gca().set_xlabel(r'Model Parameter', fontsize=13)
plt.gca().grid()
plt.gca().set_yscale('linear')

fig_bar = plt.figure(figsize=(10,5))
plt.gca().bar(cols, experiment_1.ivars.loc[pd.IndexSlice[ ivars_scale ]][cols], color='gold')
plt.gca().set_title (r'Integrated variogram Across a Range of Scales $[log-scale]$ (LgBoost_DO_124321)', fontsize = 15)
plt.gca().set_ylabel(r'IVARS-50 (Total-Variogram Effect)', fontsize = 13)
plt.gca().set_xlabel(r'Model Parameter', fontsize=13)
plt.gca().grid()
plt.gca().set_yscale('log')